# Usage example in Google Colab

In [ ]:
! pip install --upgrade transformers pymupdf qwen_vl_utils chromadb

In [ ]:
! rm -rf multimodal-rag-helper

In [ ]:
! git clone https://github.com/danielemansillo/multimodal-rag-helper.git

In [ ]:
import sys

sys.path.insert(0, "/content/multimodal-rag-helper")

In [ ]:
import json
from pathlib import Path
from typing import List

import chromadb
from IPython import display
from PIL import Image

from document_processing import DocumentProcessor, FolderProcessor
from models import DSE_Embedder

## Initialize Folder Processor

In [ ]:
folder_path = Path("/content/multimodal-rag-helper/presentations/")
folder_processor: FolderProcessor = FolderProcessor(folder_path)

## Initialize Embedder

In [ ]:
# I leave all the default values in the constructor
dse_embedder: DSE_Embedder = DSE_Embedder()

## Initialize ChromaDB

In [ ]:
# Initialize the client
client = chromadb.PersistentClient(path="chromadb")

# Create or get a collection for text and/or image embeddings
collection = client.get_or_create_collection(
    # l2 is the default
    name="my_collection", metadata={"hnsw:space": "cosine"})

## Create and save the embeddings in Chroma

In [ ]:
queries: List[str] = ["What are some good advices for the title slide?"]
queries_embedding_tensor = dse_embedder.embed_queries(queries)
# Convert to list for the search in Chroma
queries_embedding_list = queries_embedding_tensor.unsqueeze(0).tolist()

for document in folder_processor.documents:
    slides_embeddings = dse_embedder.embed_images(
        [page.image for page in document.pages], batch_size=1)
    text_embeddings = dse_embedder.embed_documents(
        [page.text for page in document.pages])

    document.set_page_embeddings(slides_embeddings)
    document.set_text_embeddings(text_embeddings)

    # Since we produced only the embeddings for the pages we only get those
    doc_embedding_records = document.get_embedding_records()
    pages_embedding_records = doc_embedding_records["page"]
    text_embedding_records = doc_embedding_records["text"]

    collection.add(**pages_embedding_records)
    collection.add(**text_embedding_records)

## Perform search and retrieval

In [ ]:
# Perform the search in the vector db
page_results = collection.query(query_embeddings=queries_embedding_list[0], n_results=3, where={"type": "page"})
text_results = collection.query(query_embeddings=queries_embedding_list[0], n_results=3, where={"type": "text"})
all_results = collection.query(query_embeddings=queries_embedding_list[0], n_results=3)

In [ ]:
# Show the page results as images
for result in page_results["metadatas"][0]:
    page_image = Image.open(result["page_path"])
    print(result)
    display.display(page_image)

In [ ]:
# Show the page results
print(json.dumps(page_results,indent=2))

In [ ]:
# Show the text results
print(json.dumps(text_results,indent=2))

In [ ]:
# Show all results
print(json.dumps(all_results,indent=2))